# Specialized Models Training

Training specialized models:
- Model 9: Formant-focused model

In [1]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.specialized.formant_focused import FormantFocusedModel
from utils.training_utils import train_model, evaluate_model
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_loader = dataloaders['feature']['train']
val_loader = dataloaders['feature']['val']
test_loader = dataloaders['feature']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)

Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model 9: Formant-focused Model

In [2]:
model9 = FormantFocusedModel(n_features=len(feature_cols), num_classes=2).to(device)
model9.set_formant_indices(feature_cols, ['formant_f1', 'formant_f2', 'formant_f3'])

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model9.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'specialized_models' / 'formant_focused'
save_dir.mkdir(parents=True, exist_ok=True)

history9, best_epoch9 = train_model(model9, train_loader, val_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='formant_focused', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model9.load_state_dict(checkpoint['model_state_dict'])
test_metrics9, _, _, _ = evaluate_model(model9, test_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics9, f, indent=2)

print(f"Model 9 Test - Acc: {test_metrics9['accuracy']:.4f}, F1: {test_metrics9['f1']:.4f}, ROC-AUC: {test_metrics9['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.2464, Train Acc: 0.8773
Val Loss: 0.2203, Val Acc: 0.8928
Val F1: 0.8958, Val ROC-AUC: 0.9757
✓ New best model saved! (F1: 0.8958)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.1901, Train Acc: 0.9101
Val Loss: 0.1933, Val Acc: 0.9196
Val F1: 0.9210, Val ROC-AUC: 0.9778
✓ New best model saved! (F1: 0.9210)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.1699, Train Acc: 0.9166
Val Loss: 0.1950, Val Acc: 0.9082
Val F1: 0.9104, Val ROC-AUC: 0.9785

Epoch 4/50
--------------------------------------------------


Train Loss: 0.1624, Train Acc: 0.9220
Val Loss: 0.1840, Val Acc: 0.9157
Val F1: 0.9174, Val ROC-AUC: 0.9810

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.0716, Train Acc: 0.9654
Val Loss: 0.2006, Val Acc: 0.9423
Val F1: 0.9426, Val ROC-AUC: 0.9832

Epoch 48/50
--------------------------------------------------


Train Loss: 0.0739, Train Acc: 0.9659
Val Loss: 0.2198, Val Acc: 0.9411
Val F1: 0.9415, Val ROC-AUC: 0.9827

Epoch 49/50
--------------------------------------------------


Train Loss: 0.0680, Train Acc: 0.9663
Val Loss: 0.2109, Val Acc: 0.9411
Val F1: 0.9414, Val ROC-AUC: 0.9830

Epoch 50/50
--------------------------------------------------


Train Loss: 0.0688, Train Acc: 0.9678
Val Loss: 0.2090, Val Acc: 0.9413
Val F1: 0.9417, Val ROC-AUC: 0.9832


Model 9 Test - Acc: 0.9400, F1: 0.9403, ROC-AUC: 0.9840
